In [81]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import pandas as pd
import os
from libcpab.libcpab.pytorch import cpab
from libcpab.libcpab.helper.utility import show_images
from IPython.display import display
import random
from sklearn.metrics import accuracy_score
import pylab as pl
from IPython import display
from skimage import io
from skimage.transform import rescale
from skimage.transform import resize
import datetime
import math
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

# Load and Construct Dataset

In [74]:
class FineGrainedDataset(Dataset):
    
    def __init__(self, csv_path, picturesize, transform=True):
        """
        Construct Fine Grained Dataset.
        
        Args
            csv_path: Path to CSV file.
            picturesize: Dimensions of sample images.
            transform: 
        """
        self.transform = transform
        self.picturesize = picturesize
        
        # construct dataframe with onehot notation
        self.df = pd.get_dummies(pd.read_csv(csv_path))
        # create ID column with image file names
        self.df['id'] = self.df['image_id'].apply(lambda x: str(x)) + "_" + self.df['tag_id'].apply(lambda x: str(x)) + ".png"
        # only use class columns and id column
        self.df = self.df.iloc[:, 10:]
        
        self.path = '/Users/mattias/Desktop/cropped/'
            
        
    def __getitem__(self, idx):
        """
        Support integer indexing on dataset (range 0 to len(self)).
        """
        sample_id = self.df['id'].iloc[idx]
        
        # get target values for sample
        targets = self.df.loc[idx, self.df.columns != 'id'].values.astype('float32')
        
        # get sample image
        image = np.asarray(Image.open(self.path + sample_id))
        
        # transform sample
        if self.transform:
            img = image
            img_normalized = np.copy(img)
            img_normalized = img_normalized / 255.
            
            img_resized = resize(img_normalized, output_shape=(self.picturesize,self.picturesize), mode='reflect', anti_aliasing=True)[:,:,:3]
            
            sample = {'id': sample_id, 'targets': torch.from_numpy(targets), 'image': torch.from_numpy(img_resized)}
        else:
            sample = {'id': sample_id, 'targets': targets, 'image': image}
        
        return sample
    
    def __len__(self):
        """
        Get length of dataset.
        """
        return len(self.df)
    

In [76]:
dataset = FineGrainedDataset(csv_path='./data/dataset_v2/train.csv', picturesize=224, transform=True)
dataset.df.head()

,sunroof,luggage_carrier,open_cargo_area,enclosed_cab,spare_wheel,wrecked,flatbed,ladder,enclosed_box,soft_shell_box,...,sub_class_van,color_black,color_blue,color_green,color_other,color_red,color_silver/grey,color_white,color_yellow,id
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,16490_15036.png
1,-1,-1,0,-1,-1,0,-1,0,1,0,...,0,0,0,0,0,1,0,0,0,16490_31658.png
2,0,0,0,0,0,0,-1,-1,-1,-1,...,0,0,0,0,0,1,0,0,0,17122_26971.png
3,0,0,0,0,0,0,-1,-1,-1,-1,...,0,0,0,0,0,0,1,0,0,12193_19301.png
4,-1,-1,0,-1,-1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,12193_35906.png


# Network

In [88]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
model_conv = models.resnet50(pretrained='imagenet')

# Disable autograd for resnet
for param in model_conv.parameters():
    param.requires_grad = False
    
# Change fully connected layer to match paper (autograd is default on new layers)
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 1024)
model_conv = model_conv.to(device)

class Net(nn.Module):

    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.output = nn.Linear(1024, num_classes)
       

    def forward(self, x):
        x = model_conv(x)
        x = F.relu(x)
        x = self.output(x)
        x = torch.sigmoid(x)
        return x


net = Net(num_classes=37).to(device)
print(net)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /Users/mattias/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [01:09<00:00, 1470333.43it/s]


Net(
  (output): Linear(in_features=1024, out_features=37, bias=True)
)


In [91]:
# test forward pass
x = np.random.uniform(0, 1, size=(10, 3, 224, 224)).astype('float32')
x = torch.autograd.Variable(torch.from_numpy(x))

out = net(x.to(device))
out.shape

torch.Size([10, 37])

# Train-validation split

In [79]:
def train_valid_split(df, trainsplit_size=0.8):
    """
    Split a dataset in training- and validation split.

    Split is made by splitting the least represented class first
    and the highest represented class last.

    Function is made for multilabel problems where samples that are already 
    drawn from a previous class are not drawn again, and the ratio of the 
    split is calculated at each class from the previous draws to match the 
    split criteria.

    Args
        df: A dataframe which the split should be created from.
            The indexes should be the ID's of the samples.
        trainsplit_size: The size of the training split. 
                         Should be a float in the interval [0, 1]
    """
    # placeholders
    cls = {}
    train_idx = []
    valid_idx = []
    
    df_cls = df.loc[:, df.columns != 'id']

    # get number of occurrences for each class
    for col in df_cls:
        size = len(df_cls.loc[df_cls[col] == 1])
        cls[col] = size

    # sort classes by ascending
    cls_sorted = sorted(cls.items(), key=lambda value: value[1])

    # iterate over classes from least represented class
    # and draw samples for training- and validation split
    for col in cls_sorted:
        # get indexes for samples where class value is 1
        indexes = df_cls.loc[df_cls[col[0]] == 1].index

        # Remove indexes that already are appended to test_idx array
        indexes = [indexes[i] for i in range(len(indexes)) if indexes[i] not in valid_idx and indexes[i] not in train_idx]

        # get size of how many indexes should be drawn for train
        train_size = int(len(indexes) * trainsplit_size)

        # get indexes for train- and validation split
        idx_train = [indexes[i] for i in sorted(random.sample(range(len(indexes)), train_size))]
        idx_valid = [indexes[i] for i in range(len(indexes)) if indexes[i] not in idx_train]

        # save indexes
        train_idx.extend(idx_train)
        valid_idx.extend(idx_valid)
        
    print("Number of samples in provided dataset: {}\n".format(len(df_cls)))
    print("Split criteria\ntrain size: {:.2f} %\nvalid size: {:.2f} %".format(trainsplit_size*100, 
                                                                   (1-trainsplit_size)*100))
    print("\nSplit results")
    print("Train size:\t{}\t({:.2f} %)".format(len(train_idx), \
                                               (len(train_idx) / len(df_cls)) * 100))
    print("Val size:\t{}\t({:.2f} %)".format(len(valid_idx), \
                                     (len(valid_idx) / len(df_cls)) * 100))

    return train_idx, valid_idx

train_indexes, valid_indexes = train_valid_split(dataset.df)

Number of samples in provided dataset: 11617

Split criteria
train size: 80.00 %
valid size: 20.00 %

Split results
Train size:	9281	(79.89 %)
Val size:	2336	(20.11 %)


# Define dataloader

In [83]:
BATCH_SIZE = 64

trainsampler = SubsetRandomSampler(train_indexes)
validsampler = SubsetRandomSampler(valid_indexes)

samplers = {'train': trainsampler,
            'valid': validsampler}

trainloader = DataLoader(dataset=dataset,
                         batch_size=BATCH_SIZE,
                         sampler=samplers['train'],
                         num_workers=4)
validloader = DataLoader(dataset=dataset,
                         batch_size=BATCH_SIZE,
                         sampler=samplers['valid'],
                         num_workers=4)

dataloaders = {'train': trainloader,
               'valid': validloader}

dataset_sizes = {x: len(samplers[x]) for x in ['train', 'valid']}

# Training Loop

In [86]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9) 

In [ ]:
epochs = 100
threshold = 0.5
valid_every = 2

losses_train = pd.DataFrame(columns=['Epoch','Loss'])
losses_valid = pd.DataFrame(columns=['Epoch','Loss'])

for epoch in range(epochs):
    batch_loss = []
    net.train()
    
    for batch in dataloaders['train']:
        inputs = batch['image'].permute(0, 3, 1, 2)
        labels = batch['targets']
        inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device, dtype=torch.float)
    
    
        # Zero the parameter gradients
        optimizer.zero_grad()
            
        # Make prediction and backpropogate
        output = net(inputs)
        loss = criterion(output,labels)
        #print(loss)
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
        
    # calculate the loss of the training set   
    losses_train.loc[epoch] = [epoch+1,np.mean(batch_loss)]
    
    if epoch % valid_every == 0 or epoch == max(range(epochs)):
        batch_loss = []
        preds = []
        trues = []
        net.eval()
        
        for batch in dataloaders['valid']:
            inputs = batch['image'].permute(0, 3, 1, 2)
            labels = batch['targets']
            inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device, dtype=torch.float)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Make prediction
            output = net(inputs)
            loss = criterion(output,labels)
            batch_loss.append(loss.item())
            
            # Append values for later calculation of accuracy
            preds.append([[1 if pred > threshold else 0 for pred in sample] for sample in output])
            trues.append([[int(value) for value in valuelist] for valuelist in labels.tolist()])
        
        # Calculate loss and accuracy for the validation set    
        losses_valid.loc[epoch] = [epoch+1,np.mean(batch_loss)]
        accuracy = MAP(preds[0], trues[0])
        
    # Plot the loss every epoch
    pl.figure(figsize=(15,5))
    pl.xlabel('Epochs')
    pl.ylabel('Loss')
    pl.title('Epoch #{}'.format(epoch+1))
    pl.plot(losses_train['Epoch'],losses_train['Loss'], '-b', label='Train')
    pl.plot(losses_valid['Epoch'],losses_valid['Loss'], '-r', label='Valid')
    pl.legend()
    pl.show()
    print('Training loss: {:.2f}'.format(losses_train['Loss'].iloc[-1],))
    print('Validation loss:{:.2f}\tValidation MAP: {:.2f}'.format(losses_valid['Loss'].iloc[-1],accuracy))
    display.clear_output(wait=True)
    

# Final plot    
pl.figure(figsize=(15,5))
pl.xlabel('Epochs')
pl.ylabel('Loss')
pl.title('Epoch #{}'.format(epoch+1))
pl.plot(losses_train['Epoch'],losses_train['Loss'], '-b', label='Train')
pl.plot(losses_valid['Epoch'],losses_valid['Loss'], '-r', label='Valid')
pl.legend()
pl.show()

print('Training loss: {:.2f}'.format(losses_train['Loss'].iloc[-1],))
print('Validation loss:{:.2f}\tValidation MAP: {:.2f}'.format(losses_valid['Loss'].iloc[-1],accuracy))

print("\n\t\t\t\t\tTraining time {}".format(datetime.datetime.now()-now))